In [ ]:
import pandas as pd
import ast
import os
import fasttext
from sklearn.metrics import classification_report, f1_score
from tqdm.auto import tqdm
from transformers import AutoTokenizer

In [ ]:
PROCESSED_DATA_PATH = '../../data/processed/'
TRAIN_BIO_FILE = PROCESSED_DATA_PATH + 'train_bio.csv'
VALIDATION_BIO_FILE = PROCESSED_DATA_PATH + 'validation_bio.csv'

FASTTEXT_DATA_PATH = '../../data/processed/fasttext/' 
TRAIN_FT_FILE = FASTTEXT_DATA_PATH + 'train.txt'

os.makedirs(FASTTEXT_DATA_PATH, exist_ok=True)

MODELS_PATH = '../../models/iteration-1/'

os.makedirs(MODELS_PATH, exist_ok=True)

MODEL_FT_FILE = '../../models/iteration-1/fasttext_baseline_v1.bin'
VALIDATION_FT_FILE = '../../data/processed/fasttext/validation.txt'

SUBMISSION_FILE = '../../data/raw/submission.csv'
OUTPUT_SUBMISSION_FILE = '../../notebooks/iteration-1/submissions/submission_fasttext_v1.csv'

In [ ]:
df_train_bio = pd.read_csv(TRAIN_BIO_FILE, sep=';')
df_validation_bio = pd.read_csv(VALIDATION_BIO_FILE, sep=';')

for col in ['tokens', 'bio_tags']:
    if isinstance(df_train_bio[col].iloc[0], str):
        print(f"Колонка '{col}' в обучающем наборе загружена как строка. Преобразуем...")
        df_train_bio[col] = df_train_bio[col].apply(ast.literal_eval)
    if isinstance(df_validation_bio[col].iloc[0], str):
        print(f"Колонка '{col}' в валидационном наборе загружена как строка. Преобразуем...")
        df_validation_bio[col] = df_validation_bio[col].apply(ast.literal_eval)

print("Данные в BIO-формате успешно загружены.")
print("\nПример данных из обучающего набора:")
display(df_train_bio.head(3))

Функция для преобразования данных в формат FastText

In [ ]:
def create_fasttext_file(df, output_path):
    num_samples = 0
    with open(output_path, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            tokens = row['tokens']
            tags = row['bio_tags']
            
            if len(tokens) != len(tags):
                print(f"Пропущена строка из-за несоответствия длин: {len(tokens)} токенов, {len(tags)} тегов.")
                continue

            for token, tag in zip(tokens, tags):
                token = str(token).replace(" ", "_")
                
                line = f"__label__{tag} {token}\n"
                f.write(line)
                num_samples += 1
    
    print(f"Файл '{output_path}' успешно создан. Записано {num_samples} обучающих примеров.")

Создание файлов

In [ ]:
create_fasttext_file(df_train_bio, TRAIN_FT_FILE)

create_fasttext_file(df_validation_bio, VALIDATION_FT_FILE)

with open(TRAIN_FT_FILE, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= 10:
            break
        print(line.strip())

Обучение модели

In [ ]:
print("Начинаем обучение модели FastText...")

try:
    model = fasttext.train_supervised(
        input=TRAIN_FT_FILE,
        epoch=25,
        lr=1.0,
        wordNgrams=2,
        dim=100,
        ws=5,
        minCount=1,
        verbose=2 
    )
    
    print("\nОбучение завершено.")
    
    model.save_model(MODEL_FT_FILE)
    print(f"Модель успешно сохранена в файл: {MODEL_FT_FILE}")
    

except Exception as e:
    print(f"Произошла ошибка во время обучения: {e}")

Проверка

In [ ]:
loaded_model = fasttext.load_model(MODEL_FT_FILE)
print("Модель успешно загружена для проверки.")

test_words = ['молоко', 'млоко', 'простоквашино', 'coca-cola', '900г', '900гр', '5%', '5проц', 'abcdefg']

print("\n--- Тестовые предсказания ---")
for word in test_words:
    prediction = loaded_model.predict(word)
    predicted_label = prediction[0][0].replace('__label__', '')
    confidence = prediction[1][0]
    print(f"Слово: '{word}' -> Предсказанный тег: '{predicted_label}' (уверенность: {confidence:.2f})")

Оценка качества обучения

In [ ]:
y_true = []
y_pred = []

print("Начинаем генерацию предсказаний для валидационной выборки...")

for line in validation_data:
    parts = line.strip().split(' ', 1)
    if len(parts) != 2:
        continue
    
    label_str, token = parts
    
    true_tag = label_str.replace('__label__', '')
    y_true.append(true_tag)
    
    prediction = model.predict(token)
    predicted_tag = prediction[0][0].replace('__label__', '')
    y_pred.append(predicted_tag)

print("Генерация предсказаний завершена.")
print(f"Обработано {len(y_true)} токенов.")

print("\nПримеры предсказаний:")
for i in range(5):
    print(f"Токен: '{validation_data[i].strip().split(' ', 1)[1]}', Истинный тег: '{y_true[i]}', Предсказанный тег: '{y_pred[i]}'")
    
labels = sorted(list(set(y_true)))

print("--- Полный отчет по классификации ---")
report = classification_report(y_true, y_pred, labels=labels, digits=4, zero_division=0)
print(report)


macro_f1 = f1_score(y_true, y_pred, labels=labels, average='macro', zero_division=0)

print("\n" + "="*50)
print(f"Итоговый Macro F1-score на валидационной выборке: {macro_f1:.4f}")
print("="*50)

Формирование файла для оффлайн-оценки

In [ ]:
TOKENIZER_NAME = 'xlm-roberta-base'
try:
    tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)
    print(f"Токенизатор '{TOKENIZER_NAME}' успешно загружен.")
except Exception as e:
    print(f"Ошибка загрузки токенизатора: {e}")


def bio_to_indices(text, bio_tags):
    if not text or not bio_tags:
        return []

    encoding = tokenizer(text, return_offsets_mapping=True, add_special_tokens=False)
    tokens_with_offsets = encoding.offset_mapping

    if len(tokens_with_offsets) != len(bio_tags):
        tokens = text.split()
        bio_tags = bio_tags[:len(tokens)] # На всякий случай
    
    annotations = []
    current_pos = 0
    for i, (token, tag) in enumerate(zip(text.split(), bio_tags)):
        start = text.find(token, current_pos)
        end = start + len(token)
        current_pos = end
        
        if tag != 'O':
            annotations.append((start, end, tag))
            
    return annotations

model = fasttext.load_model(MODEL_FT_FILE)
df_submission = pd.read_csv(SUBMISSION_FILE, sep=';')

Генерация предсказаний

In [ ]:
def predict_query(query, model):
    if not isinstance(query, str) or not query.strip():
        return []
    
    tokens = query.split()
    predicted_tags = []
    
    for token in tokens:
        prediction = model.predict(token)
        predicted_tag = prediction[0][0].replace('__label__', '')
        predicted_tags.append(predicted_tag)
        
    return predicted_tags

In [ ]:
tqdm.pandas(desc="Обработка запросов")

print("Получение BIO-тегов для всех тестовых запросов...")
predicted_bio_tags_list = df_submission['sample'].progress_apply(lambda x: predict_query(x, model))

print("Конвертация BIO-тегов в индексный формат...")
final_annotations = []
for index, row in tqdm(df_submission.iterrows(), total=len(df_submission), desc="Конвертация в индексы"):
    query_text = row['sample']
    bio_tags = predicted_bio_tags_list[index]
    if not isinstance(query_text, str):
        query_text = ""
    
    indices = bio_to_indices(query_text, bio_tags)
    final_annotations.append(indices)

df_submission['annotation'] = final_annotations

print("\nПредсказания для тестового набора успешно сгенерированы.")
display(df_submission.head())

Сохранение результатов

In [ ]:
df_submission.to_csv(OUTPUT_SUBMISSION_FILE, sep=';', index=False)

print(f"Файл для отправки успешно сохранен по пути: {OUTPUT_SUBMISSION_FILE}")

print("\nПервые 5 строк итогового файла:")
with open(OUTPUT_SUBMISSION_FILE, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= 6:
            break
        print(line.strip())